# Étude de classification humaine commune face à l'AI
<!--Auteur : corentin.boidot@ecole-navale.fr -->
Le contexte cible est celui de la reconnaissance de cible, à partir d'image radar, réalisée par deux opérateurs et un superviseur.
Dans le cadre du projet CoMAIA, on envisage l'intégration d'un logiciel avec ATR, ayant aussi le potentiel d'effectuer cette classification binaire.

Dans un premier temps (ce document), on va simplement montrer comment une telle IA entraînée à partir de données humaines peut être plus juste que chaque agent humain considéré indépendamment, mais comment la décision collective des agents (telle qu'elle nous a été rapporté en matière de GdM) peut néanmoins être plus juste que l'IA. 

On suppose ici qu'il existe une forme de consensus entre experts, qui n'est pas indépendant de la nature humaine et de ces émotions.
Nous générerons d'avance cet "humain moyen" hypothétique.
Il va nous permettre de représenter en partie ce fait que l'IA est entraînée à partir des avis de nombreux experts, et qu'elle constitue ainsi une reproduction sans hésitation de cet expert moyen, particulièrement fiable.


## Construction de la simulation

In [1]:
# Import des bibliothèques nécessaires
import numpy as np
from numpy import random
from cachetools import cached # pour l'efficacité des calculs
from tqdm import tqdm # permet d'afficher la progression des calculs sur votre écran.

In [2]:
# Modèle de "classifieur binaire", qui va nous permettre de simuler
# aussi bien la réalité terrain, que l'IA et les agents humains.
# Chacun de ces classifieurs peut être statistiquement influencé par un autre.

class VTernaryClassifier():
    '''Cette classe vise à représenter des processus de prédiction, qu'il s'agissent d'IA ou d'humain, et par analogie la réalité que les deux tentent de prédire.
    Elle est, comme eux, simulée ici de manière aléatoire.'''
    instances = {} # mémoire nécessaire pour le système d'influence.
    def __init__(self,name,correlates={},seed=None,p_neutral=None):
        '''*correlates* est un dictionnaire nom-poids, 
        où le poids de l'influence d'un avis sur l'odds_ratio de la décision.
        p_neutral représente la probabilité désirée d'indécision : 
        si l'on ne souhaite aucune indécision, laisser sur None.'''
        if seed is None:
            seed = random.randint(1000000)
        self.seed = seed
        self.correlates=correlates
        self.name = name
        self._check_integrity()
        if p_neutral is not None:
            # we turn p to odds now
            assert p_neutral<.5
            #raise Exception('Vous ne pouvez pas être aussi indécis, si ?')
            self.neutral= abs(2-1/(p_neutral))-1 # le 2 signifie les odds de base des 2 alternatives
        else:
            self.neutral = None
        VTernaryClassifier.instances[name] = self
    
    # méthode de classe, qui permet de récupérer une prédiction à partir du nom du classifieur.
    @cached(cache={})
    def get_pred(name,N=1):
        return VTernaryClassifier.instances[name].pred(N)

    def get_proba(self,i,N=1):
        '''output:
        proba_0, proba_1, proba_neutral'''
        psum = 0
        self._check_integrity()
        for k,v in self.correlates.items():
            r = 2*VTernaryClassifier.get_pred(k,N)[i]-1
            if r is not np.nan:
                psum += v*r
        odd_0 = 1+max(0,-psum)
        odd_1 = 1+max(0,psum)
        K = odd_0+odd_1
        if self.neutral is not None:
            odd_0 += self.neutral*odd_0/K
            odd_1 += self.neutral*odd_1/K
            K = odd_0+odd_1+1
        p_0, p_1 = odd_0/K, odd_1/K
        return p_0, p_1, 1-p_0-p_1
    
    @cached(cache={})
    def pred(self,N,i=None,mute=True):
        if i is not None:
            return self.pred(vdata)[i]
        res = []
        random.seed(seed=self.seed)
        gen = range(N) if mute else tqdm(range(N))
        for i in gen :
            p_0, p_1, p_nan = self.get_proba(i,N)
            dice = random.random()
            res.append(0 if dice<=p_0 else 1 if dice<=p_0+p_1 else np.nan)
        return np.array(res)
    
    def _check_integrity(self):
        if self.name in self.correlates:
            raise Exception('On ne se correle pas soi-même !')

    def _clean_instances():
        VTernaryClassifier.instances = {}

In [3]:
N=100 # on suppose qu'il y a 100 échos à classifier en mine/non-mine
truth = VTernaryClassifier('reality').pred(N)

Nous aurons un "humain moyen" avec une indécision relativement haute, ce qui mettra en exergue la liberté individuelle, puisqu'il y aura plus de lieu où c'est leur seule "influence par la vérité" qui va jouer, plutôt que l'influence par un consensus décisionnel clair.

In [4]:
humain_moyen = VTernaryClassifier('moyen',
                               correlates={'reality':6},
                               p_neutral=.1)

In [5]:
ia = VTernaryClassifier('ia',correlates={'moyen':12,'reality':1})

Nous aurons trois agents humains, plus ou moins indécis, avec un superviseur, appelé uniquement en cas de désaccord.

In [6]:
opérateur1 = VTernaryClassifier('o1',
                               correlates={'reality':1.5,'moyen':4},
                               p_neutral=.08)
opérateur2 = VTernaryClassifier('o2',
                               correlates={'reality':2,'moyen':5},
                               p_neutral=.12)
superviseur = VTernaryClassifier('sup',
                               correlates={'reality':2.5,'moyen':5},
                               p_neutral=.05)

## Mesures de justesse individuelles

In [7]:
def justesse(pred,truth):
    return np.mean(pred[~np.isnan(pred)]==truth[~np.isnan(pred)])

In [8]:
justesses = [justesse(x.pred(N),truth) for x in [ia,opérateur1,opérateur2,superviseur]]

In [9]:
justesses

[np.float64(0.71),
 np.float64(0.6989247311827957),
 np.float64(0.7263157894736842),
 np.float64(0.7346938775510204)]

Nous avons ici un cas typique où l'IA a "prouvé" sa supériorité en imitant correctement un consensus humain, relativement maîtrisé (seulement) par les opérateurs en tant que collectif.
(Ici, les indécisions ne sont pas comptées dans le calcul de la justesse.)

## L'union fait la force
On veut désormais coder le processus de prise de décision collective utilisé dans le cas de nos opérateurs de GdM. J'appelle cette procédure un "vote", en ignorant pour l'instant la capacité du superviseur à surveiller les opérateurs et à les corriger tous les deux. 
Je pars d'une inférence au faux positifs pour les cas d'hésitation des opérateurs, comme du superviseur : cela correspond à comportement attendu en matière de gestion de risque, mais qui devrait dégrader la justesse des décisions.

In [10]:
def fillna(pred,value):
    pred[np.isnan(pred)] = value
    return pred

def vote(p1,p2,p3):
    p1,p2,p3 = fillna(p1,1),fillna(p2,1),fillna(p3,1)
    return np.quantile([p1,p2,p3],q=.5,axis=0)

In [11]:
decision_commune = vote(opérateur1.pred(N),opérateur2.pred(N),superviseur.pred(N))

In [12]:
decision_commune

array([0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1.])

On n'a plus de "nan" (*Not a Number*) dans les résultats, on peut calculer leur justesse exactement.

In [13]:
justesse(decision_commune,truth)

np.float64(0.77)

In [14]:
justesses = [justesse(fillna(x.pred(N),1),truth) for x in [ia,opérateur1,opérateur2,superviseur]]

In [15]:
justesses

[np.float64(0.71), np.float64(0.68), np.float64(0.73), np.float64(0.74)]

Ici, on peut comparer les performances, avec l'inférence aux positifs.
Attention : les performances des agents sont très aléatoires, vous le verrez si vous ré-éxécutez le document.

Je vais donc répliquer l'expérience pour avoir des statistiques.

## Statistiques de performance

In [16]:
preds = []
N = 200 # longueur du rail en décision
N_simu = 200 # nb de reproduction de la simulation

#VTernaryClassifier._clean_instances()

common_truth = VTernaryClassifier('real').pred(N)

justesses = []

for i in tqdm(range(N_simu)):
    humain_moyen = VTernaryClassifier('moyen'+str(i),
                               correlates={'real':5},
                               p_neutral=.1)
    ia = VTernaryClassifier('ia'+str(i),
                               correlates={'moyen'+str(i):10,'real':1},
                               )
    opérateur1 = VTernaryClassifier('o1'+str(i),
                               correlates={'real':1.5,'moyen'+str(i):4},
                               p_neutral=.08)
    opérateur2 = VTernaryClassifier('o2'+str(i),
                               correlates={'real':2,'moyen'+str(i):5},
                               p_neutral=.12)
    superviseur = VTernaryClassifier('sup'+str(i),
                               correlates={'real':2.5,'moyen'+str(i):5},
                               p_neutral=.05)
    preds = [a.pred(N) for a in [ia,opérateur1,opérateur2,superviseur]]
    decision_commune = vote(preds[1],preds[2],preds[3])
    preds.append(decision_commune)
    justesses.append([justesse(x,common_truth) for x in preds])
    #VTernaryClassifier._clean_instances()
    #VTernaryClassifier.instances['reality'] = truth


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 234.73it/s]


In [17]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist(np.array(justesses)[:,0], bins=30,label='IA',alpha=.7)
plt.hist(np.array(justesses)[:,-1], bins=30,label='vote_humain',alpha=.7)
plt.legend()

In [18]:
np.mean(justesses,axis=0)

array([0.7821 , 0.74385, 0.75745, 0.7706 , 0.8111 ])

Ces derniers résultats montrent les performances moyennes de l'IA, dees trois agents humains, et enfin de la décision humaine collective, qui dépasse l'IA, comme le laissait voir l'histogramme.